In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [2]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [3]:
df.shape

(4026, 7)

In [4]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [6]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [9]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [10]:
y=normalized_df["Period (P)"] 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

# RandomForestRegressor

In [12]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error




def objective(trial):
  params = {
      'n_estimators': trial.suggest_int('n_estimators', 800, 5000),
      'max_depth': trial.suggest_int('max_depth', 3, 10),
      'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
      'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 20),
      
  }
  rf = RandomForestRegressor(**params, random_state=42)
  rf.fit(X_train, y_train)
  y_pred = rf.predict(X_test)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  
  return rmse
# Start time
start_time = time.time()
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)
params = study.best_params
rmse = study.best_value
print(f"Best params: {params}")
print(f"RMSE: {rmse}")

[I 2024-01-01 10:51:10,933] A new study created in memory with name: no-name-9086b186-2358-4dbb-a853-4c705f28e946
[I 2024-01-01 10:51:16,468] Trial 0 finished with value: 0.0264824931269275 and parameters: {'n_estimators': 1800, 'max_depth': 6, 'min_samples_split': 19, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.0264824931269275.
[I 2024-01-01 10:51:22,176] Trial 1 finished with value: 0.054912384751566436 and parameters: {'n_estimators': 2400, 'max_depth': 4, 'min_samples_split': 20, 'min_samples_leaf': 16}. Best is trial 0 with value: 0.0264824931269275.
[I 2024-01-01 10:51:25,629] Trial 2 finished with value: 0.020598181798083835 and parameters: {'n_estimators': 970, 'max_depth': 7, 'min_samples_split': 20, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.020598181798083835.
[I 2024-01-01 10:51:28,685] Trial 3 finished with value: 0.026772060062214373 and parameters: {'n_estimators': 1024, 'max_depth': 6, 'min_samples_split': 20, 'min_samples_leaf': 7}. Best is trial 

[I 2024-01-01 10:59:14,390] Trial 35 finished with value: 0.019541919478185236 and parameters: {'n_estimators': 4747, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 6}. Best is trial 33 with value: 0.014754021248304617.
[I 2024-01-01 10:59:27,945] Trial 36 finished with value: 0.038868208570718675 and parameters: {'n_estimators': 4967, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 33 with value: 0.014754021248304617.
[I 2024-01-01 10:59:48,439] Trial 37 finished with value: 0.01740855627636899 and parameters: {'n_estimators': 4984, 'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 33 with value: 0.014754021248304617.
[I 2024-01-01 11:00:04,599] Trial 38 finished with value: 0.02288125943425674 and parameters: {'n_estimators': 4525, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 11}. Best is trial 33 with value: 0.014754021248304617.
[I 2024-01-01 11:00:29,751] Trial 39 finished with value: 0.01475832

[I 2024-01-01 11:05:24,626] Trial 71 finished with value: 0.013059095153890754 and parameters: {'n_estimators': 1255, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 71 with value: 0.013059095153890754.
[I 2024-01-01 11:05:31,000] Trial 72 finished with value: 0.013051889893273539 and parameters: {'n_estimators': 1241, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 72 with value: 0.013051889893273539.
[I 2024-01-01 11:05:36,053] Trial 73 finished with value: 0.01575973133744988 and parameters: {'n_estimators': 1211, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 72 with value: 0.013051889893273539.
[I 2024-01-01 11:05:42,174] Trial 74 finished with value: 0.014476675211118482 and parameters: {'n_estimators': 1262, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 72 with value: 0.013051889893273539.
[I 2024-01-01 11:05:48,543] Trial 75 finished with value: 0.014476398

Total time taken:  1006.5442242622375
Best params: {'n_estimators': 1154, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 2}
RMSE: 0.011687282971855204


In [13]:
from sklearn.ensemble import RandomForestRegressor
params_best['n_estimators'] = int(params_best['n_estimators'])
params_best['max_depth'] = int(params_best['max_depth'])
params_best['min_samples_leaf'] = int(params_best['min_samples_leaf'])
rf_reg = RandomForestRegressor(**best_params)
rf_reg.fit(X_train, y_train)
print('train_mes : {:.8f}'.format(mean_squared_error(y_train, rf_reg.predict(X_train))))

train_mes : 0.00006157


In [14]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [15]:
y_pred= rf_reg.predict(X_test)
y_pred1=rf_reg.predict(X_val)
y_pred2=rf_reg.predict(X_train)

In [16]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.007863283381807043
mean_squared_error: 0.00017019809507668807
rmse: 0.01304599919809472
r2 score: 0.9965863987677182


In [17]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.004961470083045726
mean_squared_error: 6.157060602233848e-05
rmse: 0.007846693954930222
r2 score: 0.998741281113162


In [18]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.0070927944441879
mean_squared_error: 0.00013706344887048946
rmse: 0.011707409998393729
r2 score: 0.9973245818185316


In [19]:
from yellowbrick.regressor import PredictionError